# Entry & Exit Signal

- Create separate notebook as 'poc.ipynb' notebook is too bloated.
- Create abstract class


In [1]:
import pandas as pd
from abc import ABC, abstractmethod
from datetime import datetime, date
import re
from collections import deque, Counter
from decimal import Decimal
import numpy as np
import yfinance as yf
from pydantic import BaseModel, Field, computed_field, field_validator
import math
from pathlib import Path
from zoneinfo import ZoneInfo
import calendar
import itertools
from omegaconf import OmegaConf
from tqdm import tqdm

%load_ext autoreload
%autoreload 2
%matplotlib inline

from src.utils import utils
from config.variables import EntryType, CointCorrFn, STRUCT_MAPPING, EXIT_PRICE_MAPPING
from src.strategy.base.stock_trade import StockTrade

In [9]:
cfg = utils.load_config()
model_dir = f"{cfg.path.data_dir}/{cfg.date}/{cfg.single.entry_type}_{cfg.single.entry_struct}_{cfg.single.exit_struct}_{cfg.single.stop_method}/{cfg.single.hf_model}_{cfg.single.coint_corr_fn}_{cfg.single.period}"

results = utils.load_csv(
    f"{model_dir}/trade_results.csv",
    tz="America/New_York",
)
results

,news_ticker,ticker,entry_datetime,entry_action,entry_lots,entry_price,exit_datetime,exit_action,exit_lots,exit_price,days_held,profit_loss,percent_ret,daily_ret,win
0,AAPL,ANET,2025-03-27 16:00:00-04:00,buy,10,78.96,2025-03-28 16:00:00-04:00,sell,10,77.94,1.0,-1.02,-0.012918,-0.012918,0.0
1,AAPL,APH,2025-03-27 16:00:00-04:00,buy,10,68.17,2025-03-28 16:00:00-04:00,sell,10,66.59,1.0,-1.58,-0.023177,-0.023177,0.0
2,AAPL,AVGO,2025-03-27 16:00:00-04:00,buy,10,171.99,2025-03-28 16:00:00-04:00,sell,10,169.12,1.0,-2.87,-0.016687,-0.016687,0.0
3,AAPL,BRO,2025-03-27 16:00:00-04:00,buy,10,122.9,2025-03-28 16:00:00-04:00,sell,10,122.93,1.0,0.03,0.000244,0.000244,1.0
4,AAPL,COST,2025-03-27 16:00:00-04:00,buy,10,938.75,2025-03-28 16:00:00-04:00,sell,10,929.66,1.0,-9.09,-0.009683,-0.009683,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1107,XOM,VLO,2025-03-21 16:00:00-04:00,buy,10,131.51,2025-04-01 16:00:00-04:00,sell,10,133.74,11.0,2.23,0.016957,0.00153,1.0
1108,XOM,VLO,2025-03-25 16:00:00-04:00,buy,10,132.0,2025-04-01 16:00:00-04:00,sell,10,133.74,7.0,1.74,0.013182,0.001873,1.0
1109,XOM,VLO,2025-03-26 16:00:00-04:00,buy,10,134.52,2025-04-01 16:00:00-04:00,sell,10,133.74,6.0,-0.78,-0.005798,-0.000969,0.0
1110,XOM,VLO,2025-03-28 16:00:00-04:00,buy,10,131.48,2025-04-01 16:00:00-04:00,sell,10,133.74,4.0,2.26,0.017189,0.00427,1.0


In [33]:
combined = utils.load_csv(f"./data/{cfg.date}/combined_overall.csv")
combined

,entry_type,entry_struct,exit_struct,stop_method,hf_model,coint_corr_fn,period,total_num_stock_tickers,num_tickers_with_no_trades,stock_ticker_without_trades,...,last_trade,min_days_held,max_days_held,mean_days_held,median_days_held,trading_period,total_investment,total_profit,percent_return,annualized_return
0,long,multiple,fifo,no_stop,prosusai,coint,1,19,2,"NVDA, TSLA",...,2025-03-31 16:00:00-04:00,1.0,26.0,8.671811535337124,7.0,28,202954.96,531.49,0.002619,0.034684
1,long,multiple,fifo,no_stop,prosusai,coint,3,19,2,"NVDA, TSLA",...,2025-03-31 16:00:00-04:00,1.0,26.0,8.627944760357433,7.0,28,230235.3,970.92,0.004217,0.056389
2,long,multiple,fifo,no_stop,prosusai,coint,5,19,2,"NVDA, TSLA",...,2025-03-31 16:00:00-04:00,1.0,26.0,8.620967741935484,7.0,28,233255.48,332.26,0.001424,0.018723
3,long,multiple,fifo,no_stop,prosusai,pearsonr,1,19,2,"NVDA, TSLA",...,2025-03-31 16:00:00-04:00,1.0,26.0,8.635024549918167,7.0,28,300515.55,333.24,0.001109,0.014553
4,long,multiple,fifo,no_stop,prosusai,pearsonr,3,19,2,"NVDA, TSLA",...,2025-03-31 16:00:00-04:00,1.0,26.0,8.620967741935484,7.0,28,417630.01,237.11,0.000568,0.00743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5179,longshort,single,take_all,latest_loss,aventiq_ai,spearmanr,3,19,0,NaN,...,2025-03-31 16:00:00-04:00,1.0,25.0,3.927363184079602,3.0,28,302023.3,1722.13,0.005702,0.076934
5180,longshort,single,take_all,latest_loss,aventiq_ai,spearmanr,5,19,0,NaN,...,2025-03-31 16:00:00-04:00,1.0,25.0,3.993945509586277,3.0,28,315551.11,1965.04,0.006227,0.084286
5181,longshort,single,take_all,latest_loss,aventiq_ai,kendalltau,1,19,2,"MSFT, BA",...,2025-03-31 16:00:00-04:00,1.0,25.0,3.942652329749104,2.0,28,167584.5,791.14,0.004721,0.063321
5182,longshort,single,take_all,latest_loss,aventiq_ai,kendalltau,3,19,3,"TSLA, JNJ, UNH",...,2025-03-31 16:00:00-04:00,1.0,25.0,3.819377990430622,2.0,28,285606.89,1409.22,0.004934,0.066263


In [11]:
overall = utils.load_csv(f"{model_dir}/overall_summary.csv", index_col=0)
overall

,Overall Statistics
entry_type,long
entry_struct,multiple
exit_struct,take_all
stop_method,no_stop
hf_model,prosusai
coint_corr_fn,kendalltau
period,5
total_num_stock_tickers,19
num_tickers_with_no_trades,2
stock_ticker_without_trades,"NVDA, TSLA"


In [26]:
breakdown = utils.load_csv(
    f"{model_dir}/breakdown_summary.csv",
    tz="America/New_York",
)
breakdown

,news_ticker,ticker,entry_datetime_min,entry_datetime_max,exit_datetime_max,days_held_min,days_held_max,days_held_mean,days_held_median,entry_price_sum,...,win_count,win_sum,win_rate,trading_period,overall_percent_ret,overall_daily_ret,neg_ret_max,neg_ret_min,neg_ret_mean,neg_ret_median
0,AAPL,ANET,2025-03-27 16:00:00-04:00,2025-03-27 16:00:00-04:00,2025-03-28 16:00:00-04:00,1.0,1.0,1.0,1.0,78.96,...,1,0.0,0.0,1,-0.012918,-0.012918,-0.012918,-0.012918,-0.012918,-0.012918
1,AAPL,APH,2025-03-27 16:00:00-04:00,2025-03-27 16:00:00-04:00,2025-03-28 16:00:00-04:00,1.0,1.0,1.0,1.0,68.17,...,1,0.0,0.0,1,-0.023177,-0.023177,-0.023177,-0.023177,-0.023177,-0.023177
2,AAPL,AVGO,2025-03-27 16:00:00-04:00,2025-03-27 16:00:00-04:00,2025-03-28 16:00:00-04:00,1.0,1.0,1.0,1.0,171.99,...,1,0.0,0.0,1,-0.016687,-0.016687,-0.016687,-0.016687,-0.016687,-0.016687
3,AAPL,BRO,2025-03-27 16:00:00-04:00,2025-03-27 16:00:00-04:00,2025-03-28 16:00:00-04:00,1.0,1.0,1.0,1.0,122.9,...,1,1.0,1.0,1,0.000244,0.000244,NaN,NaN,NaN,NaN
4,AAPL,COST,2025-03-27 16:00:00-04:00,2025-03-27 16:00:00-04:00,2025-03-28 16:00:00-04:00,1.0,1.0,1.0,1.0,938.75,...,1,0.0,0.0,1,-0.009683,-0.009683,-0.009683,-0.009683,-0.009683,-0.009683
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,XOM,MPC,2025-03-03 16:00:00-05:00,2025-03-31 16:00:00-04:00,2025-04-01 16:00:00-04:00,1.0,13.0,5.153846153846154,4.0,1856.73,...,13,7.0,0.538462,28,0.010955,0.000389,-0.056714,-0.000073,-0.022729,-0.020442
154,XOM,PSX,2025-03-03 16:00:00-05:00,2025-03-31 16:00:00-04:00,2025-04-01 16:00:00-04:00,1.0,13.0,5.153846153846154,4.0,1616.07,...,13,9.0,0.692308,28,0.015674,0.000556,-0.036047,-0.009084,-0.017939,-0.013312
155,XOM,TKO,2025-03-03 16:00:00-05:00,2025-03-31 16:00:00-04:00,2025-04-01 16:00:00-04:00,1.0,13.0,5.153846153846154,4.0,1929.04,...,13,7.0,0.538462,28,0.000746,0.000027,-0.056291,-0.008443,-0.022546,-0.016324
156,XOM,TRGP,2025-03-03 16:00:00-05:00,2025-03-31 16:00:00-04:00,2025-04-01 16:00:00-04:00,1.0,13.0,5.153846153846154,4.0,2508.12,...,13,9.0,0.692308,28,0.016917,0.000599,-0.07037,-0.000385,-0.027287,-0.019196


In [17]:
breakdown.columns

Index(['news_ticker', 'ticker', 'entry_datetime_min', 'entry_datetime_max',
       'exit_datetime_max', 'days_held_min', 'days_held_max', 'days_held_mean',
       'days_held_median', 'entry_price_sum', 'profit_loss_sum',
       'percent_ret_min', 'percent_ret_max', 'percent_ret_mean',
       'percent_ret_median', 'win_count', 'win_sum', 'trading_period',
       'overall_percent_ret', 'overall_daily_ret', 'neg_ret_max',
       'neg_ret_min', 'neg_ret_mean', 'neg_ret_median'],
      dtype='object')

In [31]:
df_top_n = utils.load_csv("./data/2025-04-01/top_15_tickers.csv")
df_top_n

,strategy,entry_type,entry_struct,exit_struct,stop_method,hf_model,coint_corr_fn,period,ticker_pair,news_ticker,ticker,overall_daily_ret,days_held_max,trading_period,win_rate,win_count,neg_ret_mean,neg_ret_max
0,long_multiple_fifo_no_stop_prosusai_coint_1,long,multiple,fifo,no_stop,prosusai,coint,1,GOOGL_SJM,GOOGL,SJM,0.002978,2.0,8,1.0,3,NaN,NaN
1,long_multiple_fifo_no_stop_prosusai_coint_1,long,multiple,fifo,no_stop,prosusai,coint,1,AMZN_CI,AMZN,CI,0.002889,5.0,5,1.0,3,NaN,NaN
2,long_multiple_fifo_no_stop_prosusai_coint_1,long,multiple,fifo,no_stop,prosusai,coint,1,GOOGL_MOH,GOOGL,MOH,0.002865,2.0,8,0.666667,3,-0.0051,-0.0051
3,long_multiple_fifo_no_stop_prosusai_coint_1,long,multiple,fifo,no_stop,prosusai,coint,1,GOOGL_MDLZ,GOOGL,MDLZ,0.002211,2.0,8,0.666667,3,-0.002948,-0.002948
4,long_multiple_fifo_no_stop_prosusai_coint_1,long,multiple,fifo,no_stop,prosusai,coint,1,UNH_ULTA,UNH,ULTA,0.001991,14.0,28,0.714286,7,-0.008486,-0.015508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77755,longshort_single_take_all_latest_loss_aventiq_...,longshort,single,take_all,latest_loss,aventiq_ai,kendalltau,5,MSFT_DELL,MSFT,DELL,0.001422,7.0,12,0.666667,3,-0.018399,-0.018399
77756,longshort_single_take_all_latest_loss_aventiq_...,longshort,single,take_all,latest_loss,aventiq_ai,kendalltau,5,NVDA_ANET,NVDA,ANET,0.001188,5.0,5,1.0,1,NaN,NaN
77757,longshort_single_take_all_latest_loss_aventiq_...,longshort,single,take_all,latest_loss,aventiq_ai,kendalltau,5,MSFT_KKR,MSFT,KKR,0.001181,7.0,12,1.0,3,NaN,NaN
77758,longshort_single_take_all_latest_loss_aventiq_...,longshort,single,take_all,latest_loss,aventiq_ai,kendalltau,5,META_LII,META,LII,0.001108,11.0,14,0.666667,3,-0.008287,-0.008287


In [32]:
df_pivot_top_n = utils.load_csv("./data/2025-04-01/pivot_top_15_tickers.csv")
df_pivot_top_n

,strategy,top_1,top_2,top_3,top_4,top_5,top_6,top_7,top_8,top_9,top_10,top_11,top_12,top_13,top_14,top_15
0,long_multiple_fifo_no_stop_prosusai_coint_1,AMZN_CI,GOOGL_CI,GOOGL_MDLZ,GOOGL_MOH,GOOGL_SJM,GOOGL_XOM,HD_MSI,JNJ_CDNS,JNJ_EOG,JPM_CTVA,META_CTVA,MSFT_EOG,NFLX_BR,UNH_ULTA,XOM_SJM
1,long_multiple_fifo_no_stop_prosusai_coint_3,AMZN_BR,AMZN_NRG,AMZN_WM,HD_EOG,JNJ_CTRA,JNJ_CVX,JNJ_NEM,META_ADM,META_CZR,NFLX_AZO,UNH_NOC,V_DVN,WMT_DLTR,WMT_EOG,XOM_HCA
2,long_multiple_fifo_no_stop_prosusai_coint_5,AMZN_DLR,AMZN_DLTR,AMZN_EA,AMZN_ECL,BA_JKHY,HD_CAG,HD_EA,JNJ_KDP,META_CAG,PG_TSCO,UNH_NOC,XOM_CI,XOM_EA,XOM_ED,XOM_NOC
3,long_multiple_fifo_no_stop_prosusai_pearsonr_1,AMZN_COF,AMZN_DFS,AMZN_TSLA,BA_DG,BA_DLTR,BA_DPZ,JNJ_MMC,JPM_BR,JPM_DFS,META_LYV,META_MA,MSFT_ROST,NFLX_FOX,V_DRI,XOM_CI
4,long_multiple_fifo_no_stop_prosusai_pearsonr_3,AMZN_BR,AMZN_COST,AMZN_CRWD,AMZN_NOW,BA_SLB,META_MA,MSFT_CMG,MSFT_CPRT,PG_HIG,UNH_EA,UNH_NOC,XOM_ACGL,XOM_FANG,XOM_HES,XOM_PCG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5179,longshort_single_take_all_latest_loss_aventiq_...,AAPL_AVGO,BA_ON,MSFT_CMG,MSFT_DELL,MSFT_GOOG,MSFT_GOOGL,MSFT_META,MSFT_NOW,NVDA_AVGO,NVDA_BKNG,NVDA_GE,NVDA_META,NVDA_PNR,NVDA_RCL,TSLA_GNRC
5180,longshort_single_take_all_latest_loss_aventiq_...,AAPL_AVGO,BAC_CBRE,JPM_KKR,META_LII,META_MAS,META_PNR,MSFT_AAPL,MSFT_DELL,NVDA_APH,NVDA_APO,NVDA_AVGO,NVDA_ETN,NVDA_MSFT,NVDA_NVR,NVDA_ORCL
5181,longshort_single_take_all_latest_loss_aventiq_...,JPM_DFS,JPM_WMT,META_COST,META_IBM,META_TKO,META_WMT,NVDA_ANET,NVDA_APH,NVDA_HPE,NVDA_LII,NVDA_ORCL,NVDA_TT,TSLA_CCL,TSLA_GLW,V_DRI
5182,longshort_single_take_all_latest_loss_aventiq_...,META_PNR,MSFT_CMG,MSFT_DELL,MSFT_GOOG,MSFT_GOOGL,MSFT_META,MSFT_NOW,NVDA_ANET,NVDA_AVGO,NVDA_BKNG,NVDA_GE,NVDA_META,NVDA_PNR,NVDA_RCL,V_HWM


In [4]:
corr = pd.read_csv("./data/coint_corr/2025-04-01/coint_corr_1y.csv")
corr

,ticker1,ticker2,coint,pearsonr,spearmanr,kendalltau
0,MMM,AOS,0.318636,-0.739837,-0.674401,-0.504392
1,MMM,ABT,0.037253,0.844638,0.841827,0.605854
2,MMM,ABBV,0.247177,0.795093,0.763711,0.565531
3,MMM,ACN,0.701204,0.713788,0.675311,0.486818
4,MMM,ADBE,0.603408,-0.332484,-0.339570,-0.217770
...,...,...,...,...,...,...
123251,YUM,ZBH,0.958000,-0.032467,0.140320,0.094599
123252,YUM,ZTS,0.864792,-0.295493,-0.281127,-0.178846
123253,ZBRA,ZBH,0.481884,-0.410229,-0.326750,-0.203995
123254,ZBRA,ZTS,0.847695,0.377118,0.407230,0.289736


In [7]:
snp500_list = utils.gen_snp500_list(cfg.snp500_url, cfg.ignore_list)
snp500_list

['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ACN',
 'ADBE',
 'AMD',
 'AES',
 'AFL',
 'A',
 'APD',
 'ABNB',
 'AKAM',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AEE',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'AON',
 'APA',
 'APO',
 'AAPL',
 'AMAT',
 'APTV',
 'ACGL',
 'ADM',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'AXON',
 'BKR',
 'BALL',
 'BAC',
 'BAX',
 'BDX',
 'BBY',
 'TECH',
 'BIIB',
 'BLK',
 'BX',
 'BK',
 'BA',
 'BKNG',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BLDR',
 'BG',
 'BXP',
 'CHRW',
 'CDNS',
 'CZR',
 'CPT',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'COR',
 'CNC',
 'CNP',
 'CF',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CSCO',
 'C',
 'CFG',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMCSA',
 'CAG',
 'COP',
 'ED',
 'STZ',
 'CEG',
 'COO',
 'CPRT',
 'GLW',
 'CPAY',
 '

In [14]:
ticker_dict = {period: [] for period in [1, 3, 5]}
for ticker in snp500_list:
    for period in [1, 3, 5]:
        corr = pd.read_csv(f"./data/coint_corr/2025-04-01/coint_corr_{period}y.csv")
        temp = corr.loc[
            ((corr["ticker1"] == ticker) | (corr["ticker2"] == ticker))
            & (corr["kendalltau"] > 0.5),
            :,
        ]
        if temp.empty:
            ticker_dict[period].append(ticker)

ticker_dict

{1: ['A',
  'AIG',
  'BDX',
  'TECH',
  'BA',
  'CSGP',
  'CVS',
  'EOG',
  'GEV',
  'HSIC',
  'INTU',
  'MSFT',
  'PARA',
  'ROST',
  'SW',
  'SOLV',
  'STLD',
  'SYY',
  'TEL',
  'YUM'],
 3: ['BDX',
  'CHTR',
  'CVX',
  'STZ',
  'EW',
  'F',
  'GEHC',
  'GEV',
  'GPN',
  'GL',
  'IEX',
  'INTC',
  'JNJ',
  'KVUE',
  'LULU',
  'MKTX',
  'MTD',
  'MOH',
  'NOC',
  'PEP',
  'PLD',
  'SW',
  'SOLV',
  'TSLA',
  'UNH',
  'VLTO'],
 5: ['APD',
  'ABNB',
  'AKAM',
  'T',
  'BDX',
  'BIIB',
  'CPB',
  'CAG',
  'CEG',
  'CSGP',
  'DXCM',
  'DASH',
  'EXE',
  'GEHC',
  'GEV',
  'HOLX',
  'PODD',
  'JKHY',
  'KVUE',
  'KMB',
  'LULU',
  'PLTR',
  'SW',
  'SOLV',
  'SBUX',
  'VLTO',
  'VRSN']}

In [16]:
for k, v in ticker_dict.items():
    print(f"{k} : {len(v)}")

1 : 20
3 : 26
5 : 27


In [ ]:
{
    k: [ticker for ticker in v if ticker in cfg.stock_list]
    for k, v in ticker_dict.items()
}

{1: ['BA', 'MSFT'], 3: ['JNJ', 'TSLA', 'UNH'], 5: []}

In [118]:
top_n = utils.load_csv(
    f"{model_dir}/top_ticker_pairs.csv",
    tz="America/New_York",
)
top_n

,news_ticker,ticker,entry_datetime_min,entry_datetime_max,exit_datetime_max,days_held_min,days_held_max,days_held_mean,days_held_median,entry_price_sum,...,daily_ret_median,win_count,win_sum,trading_period,overall_percent_ret,overall_daily_ret,neg_ret_max,neg_ret_min,neg_ret_mean,neg_ret_median
0,NVDA,APH,2025-03-27 16:00:00-04:00,2025-03-27 16:00:00-04:00,2025-04-01 16:00:00-04:00,5,5,5.0,5.0,68.17,...,0.006086,1,1,5,0.030805,0.006086,NaN,NaN,NaN,NaN
1,MSFT,CMG,2025-03-20 16:00:00-04:00,2025-03-31 16:00:00-04:00,2025-04-01 16:00:00-04:00,1,7,4.0,4.0,150.13,...,0.005506,3,3,12,0.027243,0.002242,NaN,NaN,NaN,NaN
2,AAPL,AVGO,2025-03-20 16:00:00-04:00,2025-03-31 16:00:00-04:00,2025-04-01 16:00:00-04:00,1,7,4.0,4.0,529.96,...,-0.00651,3,1,12,0.024341,0.002006,-0.026513,-0.00651,-0.016511,-0.016511
3,META,LII,2025-03-18 16:00:00-04:00,2025-03-21 16:00:00-04:00,2025-04-01 16:00:00-04:00,1,11,4.666667,2.0,1742.35,...,0.017304,3,2,14,0.015623,0.001108,-0.008287,-0.008287,-0.008287,-0.008287
4,BAC,CBRE,2025-03-03 16:00:00-05:00,2025-03-26 16:00:00-04:00,2025-04-01 16:00:00-04:00,1,7,4.0,5.0,926.24,...,0.005956,7,5,28,0.031385,0.001104,-0.019159,-0.01214,-0.01565,-0.01565
5,JPM,KKR,2025-03-03 16:00:00-05:00,2025-03-27 16:00:00-04:00,2025-04-01 16:00:00-04:00,1,11,3.5,1.5,959.74,...,0.0054565,8,5,28,0.028706,0.001011,-0.0385,-0.001722,-0.020883,-0.022426
6,JNJ,KDP,2025-03-03 16:00:00-05:00,2025-03-26 16:00:00-04:00,2025-04-01 16:00:00-04:00,1,15,7.0,6.0,134.56,...,0.0034995,4,3,28,0.026457,0.000933,-0.006799,-0.006799,-0.006799,-0.006799
7,V,GE,2025-03-03 16:00:00-05:00,2025-03-06 16:00:00-05:00,2025-04-01 16:00:00-04:00,1,25,9.333333,2.0,599.39,...,0.001624,3,3,28,0.025142,0.000887,NaN,NaN,NaN,NaN
8,UNH,NOC,2025-03-03 16:00:00-05:00,2025-03-31 16:00:00-04:00,2025-04-01 16:00:00-04:00,1,15,4.0,2.0,3419.19,...,0.002129,7,6,28,0.023985,0.000847,-0.002109,-0.002109,-0.002109,-0.002109
9,BA,PPG,2025-03-10 16:00:00-04:00,2025-03-28 16:00:00-04:00,2025-04-01 16:00:00-04:00,1,6,3.666667,3.5,671.52,...,0.003995,6,5,22,0.016708,0.000753,-0.012025,-0.012025,-0.012025,-0.012025


In [122]:
from src.cal_profit_loss import CalProfitLoss

# Get list of combinations for long, short and long-short strategies
combi_list = [list(itertools.product(*strat)) for strat in cfg.full]
combi_list = [combi for sub_list in combi_list for combi in sub_list]

for (
    ent_type,
    ent_struct,
    ex_struct,
    stop_method,
    hf_model,
    coint_corr_fn,
    period,
) in tqdm(combi_list):
    # model_dir = f"./data/2025-04-01/{ent_type}_{ent_struct}_{ex_struct}_{stop_method}/{hf_model}_{coint_corr_fn}_{period}"
    # results = utils.load_csv(f"{model_dir}/trade_results.csv", tz="America/New_York")
    # breakdown = utils.load_csv(
    #     f"{model_dir}/breakdown_summary.csv", tz="America/New_York"
    # )

    # breakdown = append_neg_rets(results, breakdown)
    # utils.save_csv(breakdown, f"{model_dir}/breakdown_summary.csv", save_index=True)
    cal_pl = CalProfitLoss(
        path=cfg.path,
        no_trades=[],
        date=cfg.date,
        entry_type=ent_type,
        entry_struct=ent_struct,
        exit_struct=ex_struct,
        stop_method=stop_method,
        hf_model=hf_model,
        coint_corr_fn=coint_corr_fn,
        period=period,
    )
    _ = cal_pl.gen_breakdown_summary()

100%|██████████| 5184/5184 [12:18<00:00,  7.02it/s]


# Table of Contents

1. [GenTrades](#gentrades)


In [14]:
trade = StockTrade(
    ticker="AAPL",
    entry_datetime="2025-04-01",
    entry_action="buy",
    entry_lots=10,
    entry_price=200,
)
trade1 = StockTrade(
    ticker="AAPL",
    entry_datetime="2025-04-02",
    entry_action="buy",
    entry_lots=10,
    entry_price=195,
)

open_trades = deque([trade, trade1])
class_name = STRUCT_MAPPING.get("multiple")
entry_method = utils.get_class_instance(
    class_name, "./src/strategy/base/entry_struct.py", num_lots=20
)
open_trades = entry_method.open_new_pos(open_trades, "AAPL", "2025-4-4", "buy", 192)
open_trades

deque([StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 1, 0, 0), entry_action='buy', entry_lots=Decimal('10'), entry_price=Decimal('200'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None),
       StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 2, 0, 0), entry_action='buy', entry_lots=Decimal('10'), entry_price=Decimal('195'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None),
       StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 4, 0, 0), entry_action='buy', entry_lots=Decimal('20'), entry_price=Decimal('192'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None)])

In [19]:
utils.display_open_trades(open_trades)

open_trades : 
[
   ticker: 'AAPL', ent_dt: '2025-04-01', ent_act: 'buy', ent_lots: 10, ent_price: 200, ex_dt: 'None', ex_act: 'None', ex_lots: 0, ex_price: None
   ticker: 'AAPL', ent_dt: '2025-04-02', ent_act: 'buy', ent_lots: 10, ent_price: 195, ex_dt: 'None', ex_act: 'None', ex_lots: 0, ex_price: None
   ticker: 'AAPL', ent_dt: '2025-04-04', ent_act: 'buy', ent_lots: 20, ent_price: 192, ex_dt: 'None', ex_act: 'None', ex_lots: 0, ex_price: None
]



In [140]:
open_trades = deque()
class_name = STRUCT_MAPPING.get("multiple_half")
entry_method = utils.get_class_instance(
    class_name, "./src/strategy/base/entry_struct.py", num_lots=20
)
open_trades = entry_method.open_new_pos(open_trades, "AAPL", "2025-04-01", "sell", 192)
open_trades = entry_method.open_new_pos(open_trades, "AAPL", "2025-04-02", "sell", 196)
open_trades = entry_method.open_new_pos(open_trades, "AAPL", "2025-04-03", "sell", 193)
open_trades

deque([StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 1, 0, 0), entry_action='sell', entry_lots=Decimal('20'), entry_price=Decimal('192'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None),
       StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 2, 0, 0), entry_action='sell', entry_lots=Decimal('10'), entry_price=Decimal('196'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None),
       StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 3, 0, 0), entry_action='sell', entry_lots=Decimal('5'), entry_price=Decimal('193'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None)])

In [50]:
open_trades

deque([StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 1, 0, 0), entry_action='buy', entry_lots=Decimal('10'), entry_price=Decimal('200'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None),
       StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 2, 0, 0), entry_action='buy', entry_lots=Decimal('10'), entry_price=Decimal('195'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None),
       StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 4, 0, 0), entry_action='buy', entry_lots=Decimal('20'), entry_price=Decimal('192'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None)])

In [51]:
class_name = STRUCT_MAPPING.get("half_lifo")
exit_method = utils.get_class_instance(class_name, "./src/strategy/base/exit_struct.py")
open_trades, completed_trades = exit_method.close_pos(
    open_trades, "2025-04-10", "buy", 180
)
open_trades

Validation Error : 1 validation error for StockTrade
exit_action
  Value error, Entry action cannot be the same as exit action except when both are 'wait'. [type=value_error, input_value='buy', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error


deque([StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 1, 0, 0), entry_action='buy', entry_lots=Decimal('10'), entry_price=Decimal('200'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None),
       StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 2, 0, 0), entry_action='buy', entry_lots=Decimal('10'), entry_price=Decimal('195'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None),
       StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 4, 0, 0), entry_action='buy', entry_lots=Decimal('20'), entry_price=Decimal('192'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None)])

In [131]:
open_trades = deque()
class_name = STRUCT_MAPPING.get("multiple_half")
entry_method = utils.get_class_instance(
    class_name, "./src/strategy/base/entry_struct.py", num_lots=20
)
open_trades = entry_method.open_new_pos(open_trades, "AAPL", "2025-04-01", "buy", 192)
open_trades = entry_method.open_new_pos(open_trades, "AAPL", "2025-04-02", "buy", 196)
open_trades = entry_method.open_new_pos(open_trades, "AAPL", "2025-04-03", "buy", 193)
open_trades

deque([StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 1, 0, 0), entry_action='buy', entry_lots=Decimal('20'), entry_price=Decimal('192'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None),
       StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 2, 0, 0), entry_action='buy', entry_lots=Decimal('10'), entry_price=Decimal('196'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None),
       StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 3, 0, 0), entry_action='buy', entry_lots=Decimal('5'), entry_price=Decimal('193'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None)])

In [141]:
class_name = EXIT_PRICE_MAPPING.get("percent_loss")
stop_price = utils.get_class_instance(
    class_name, "./src/strategy/base/cal_exit_price.py", percent_loss=0.1
)
stop_price.cal_exit_price(open_trades)

Decimal('212.61')

In [30]:
pos = "long"
trade1 = (10, 141.87)
trade2 = (10, 141.97)
trade3 = (10, 143.25)
percent_loss = 0.001
trades = [
    trade1,
    trade2,
    trade3,
]
total_lots = sum(trade[0] for trade in trades)
total_invest = sum(trade[0] * trade[1] for trade in trades)
# stop_loss = [trade[1] * (1-percent_loss) if pos == "long" else trade[1] * (1+percent_loss) for trade in trades]
# stop_loss = [Decimal(str(round(loss, 2))) for loss in stop_loss]
# stop_loss

# (10 * 200 + 10 * 195 + 20 * 192 ) * (1-percent_loss)/ 40

stop_price = (
    total_invest * (1 - percent_loss) / total_lots
    if pos == "long"
    else total_invest * (1 + percent_loss) / total_lots
)
stop_price

142.22096999999997

In [32]:
142.9 * (1 - percent_loss)

142.7571

# [GenTrades](#table-of-contents)


In [58]:
date = "2025-04-01"
entry_type = "longshort"
entry_struct = "multiple"
exit_struct = "take_all"
stop_method = "no_stop"
model = "ziweichen_coint_5"

date_dir = f"./data/{date}"
results_dir = f"{date_dir}/{entry_type}_{entry_struct}_{exit_struct}_{stop_method}"
model_dir = f"{results_dir}/{model}"
pa_dir = f"{model_dir}/price_actions"

In [67]:
corr = pd.read_csv("./data/coint_corr/2025-04-01/coint_corr_1y.csv")
corr

,ticker1,ticker2,coint,pearsonr,spearmanr,kendalltau
0,MMM,AOS,0.318636,-0.739837,-0.674401,-0.504392
1,MMM,ABT,0.037253,0.844638,0.841827,0.605854
2,MMM,ABBV,0.247177,0.795093,0.763711,0.565531
3,MMM,ACN,0.701204,0.713788,0.675311,0.486818
4,MMM,ADBE,0.603408,-0.332484,-0.339570,-0.217770
...,...,...,...,...,...,...
123251,YUM,ZBH,0.958000,-0.032467,0.140320,0.094599
123252,YUM,ZTS,0.864792,-0.295493,-0.281127,-0.178846
123253,ZBRA,ZBH,0.481884,-0.410229,-0.326750,-0.203995
123254,ZBRA,ZTS,0.847695,0.377118,0.407230,0.289736


In [70]:
pd.concat([corr, pd.DataFrame()], axis=0)

,ticker1,ticker2,coint,pearsonr,spearmanr,kendalltau
0,MMM,AOS,0.318636,-0.739837,-0.674401,-0.504392
1,MMM,ABT,0.037253,0.844638,0.841827,0.605854
2,MMM,ABBV,0.247177,0.795093,0.763711,0.565531
3,MMM,ACN,0.701204,0.713788,0.675311,0.486818
4,MMM,ADBE,0.603408,-0.332484,-0.339570,-0.217770
...,...,...,...,...,...,...
123251,YUM,ZBH,0.958000,-0.032467,0.140320,0.094599
123252,YUM,ZTS,0.864792,-0.295493,-0.281127,-0.178846
123253,ZBRA,ZBH,0.481884,-0.410229,-0.326750,-0.203995
123254,ZBRA,ZTS,0.847695,0.377118,0.407230,0.289736


In [59]:
results = utils.load_csv(f"{model_dir}/trade_results.csv", tz="America/New_York")
results

,news_ticker,ticker,entry_datetime,entry_action,entry_lots,entry_price,exit_datetime,exit_action,exit_lots,exit_price,days_held,profit_loss,percent_ret,daily_ret,win
0,AAPL,ANET,2025-03-20 16:00:00-04:00,sell,10,82.97,2025-04-01 16:00:00-04:00,buy,10,78.49,12,4.48,0.053995,0.004392,1
1,AAPL,ANET,2025-03-21 16:00:00-04:00,sell,10,83.13,2025-04-01 16:00:00-04:00,buy,10,78.49,11,4.64,0.055816,0.00495,1
2,AAPL,ANET,2025-03-24 16:00:00-04:00,sell,10,87.51,2025-04-01 16:00:00-04:00,buy,10,78.49,8,9.02,0.103074,0.012338,1
3,AAPL,ANET,2025-03-25 16:00:00-04:00,sell,10,86.94,2025-04-01 16:00:00-04:00,buy,10,78.49,7,8.45,0.097193,0.013339,1
4,AAPL,ANET,2025-03-26 16:00:00-04:00,sell,10,81.66,2025-04-01 16:00:00-04:00,buy,10,78.49,6,3.17,0.038819,0.006368,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1965,XOM,SWKS,2025-03-25 16:00:00-04:00,buy,10,68.22,2025-03-27 16:00:00-04:00,sell,10,66.3,2,-1.92,-0.028144,-0.014172,0
1966,XOM,SWKS,2025-03-26 16:00:00-04:00,buy,10,67.6,2025-03-27 16:00:00-04:00,sell,10,66.3,1,-1.3,-0.019231,-0.019231,0
1967,XOM,SWKS,2025-03-27 16:00:00-04:00,sell,10,66.3,2025-03-28 16:00:00-04:00,buy,10,64.12,1,2.18,0.032881,0.032881,1
1968,XOM,SWKS,2025-03-28 16:00:00-04:00,buy,10,64.12,2025-04-01 16:00:00-04:00,sell,10,64.62,4,0.5,0.007798,0.001944,1


In [64]:
ticker_pair = "TSLA_CAG"
news_ticker = ticker_pair.split("_")[0]
ticker = ticker_pair.split("_")[1]
results.loc[(results["news_ticker"] == news_ticker) & (results["ticker"] == ticker), :]

,news_ticker,ticker,entry_datetime,entry_action,entry_lots,entry_price,exit_datetime,exit_action,exit_lots,exit_price,days_held,profit_loss,percent_ret,daily_ret,win
1426,TSLA,CAG,2025-03-27 16:00:00-04:00,buy,10,26.46,2025-03-28 16:00:00-04:00,sell,10,26.55,1,0.09,0.003401,0.003401,1
1427,TSLA,CAG,2025-03-28 16:00:00-04:00,sell,10,26.55,2025-04-01 16:00:00-04:00,buy,10,26.6,4,-0.05,-0.001883,-0.000471,0
1428,TSLA,CAG,2025-03-31 16:00:00-04:00,sell,10,26.67,2025-04-01 16:00:00-04:00,buy,10,26.6,1,0.07,0.002625,0.002625,1


In [65]:
df = utils.load_csv(f"{pa_dir}/{ticker_pair}.csv")
df.loc[
    ~df["median_rating_excl"].isna(),
    [
        "date",
        "coint_corr_ticker",
        "close",
        "ticker",
        "median_rating_excl",
        "entry_signal",
        "exit_signal",
    ],
]

,date,coint_corr_ticker,close,ticker,median_rating_excl,entry_signal,exit_signal
0,2025-03-27,CAG,26.46,TSLA,4.0,buy,buy
1,2025-03-28,CAG,26.55,TSLA,1.0,sell,sell
2,2025-03-31,CAG,26.67,TSLA,1.0,sell,sell
3,2025-04-01,CAG,26.6,TSLA,1.0,sell,sell


In [134]:
entry_type = "long_only"
# entry_type = "short_only"
# entry_type = "long_or_short"

entry = utils.get_class_instance(
    "SentiEntry",
    "./src/strategy/entry/senti_entry.py",
    entry_type=entry_type,
)
exit = utils.get_class_instance(
    "SentiExit",
    "./src/strategy/exit/senti_exit.py",
    entry_type=entry_type,
)

df_signal = entry.gen_entry_signal(df)
df_signal = exit.gen_exit_signal(df_signal)
df_signal

,date,coint_corr_ticker,open,high,low,close,volume,ticker,day_name,av_rating,median_rating_excl,entry_signal,exit_signal
0,2025-03-03,EA,130.38,131.6,128.87,129.23,5209500,NFLX,Monday,3.0,3,wait,wait
1,2025-03-04,EA,129.52,133.35,129.24,131.82,4291000,NFLX,Tuesday,3.0,3,wait,wait
2,2025-03-05,EA,131.6,134.51,131.31,134.05,5218000,NFLX,Wednesday,3.25,4,buy,wait
3,2025-03-06,EA,133.65,137.53,133.37,136.79,3676700,NFLX,Thursday,4.5,5,buy,wait
4,2025-03-07,EA,136.19,140.42,136.0,140.04,3356800,NFLX,Friday,3.0,3,wait,wait
5,2025-03-10,EA,139.75,142.97,139.22,140.43,5440900,NFLX,Monday,5.0,5,buy,wait
6,2025-03-11,EA,140.9,140.9,136.9,137.88,3837200,NFLX,Tuesday,3.5,4,buy,wait
7,2025-03-12,EA,137.0,138.15,135.73,136.12,3177500,NFLX,Wednesday,3.0,3,wait,wait
8,2025-03-13,EA,135.79,138.26,135.23,137.72,3165100,NFLX,Thursday,3.1666666666666665,3,wait,wait
9,2025-03-14,EA,138.15,139.23,137.87,138.71,3065500,NFLX,Friday,3.0,3,wait,wait


In [165]:
df_signal

,date,coint_corr_ticker,open,high,low,close,volume,ticker,day_name,av_rating,median_rating_excl,entry_signal,exit_signal
0,2025-03-03,EA,130.38,131.6,128.87,129.23,5209500,NFLX,Monday,3.0,3,wait,wait
1,2025-03-04,EA,129.52,133.35,129.24,131.82,4291000,NFLX,Tuesday,3.0,3,wait,wait
2,2025-03-05,EA,131.6,134.51,131.31,134.05,5218000,NFLX,Wednesday,3.25,4,buy,wait
3,2025-03-06,EA,133.65,137.53,133.37,136.79,3676700,NFLX,Thursday,4.5,5,buy,wait
4,2025-03-07,EA,136.19,140.42,136.0,140.04,3356800,NFLX,Friday,3.0,3,wait,wait
5,2025-03-10,EA,139.75,142.97,139.22,140.43,5440900,NFLX,Monday,5.0,5,buy,wait
6,2025-03-11,EA,140.9,140.9,136.9,137.88,3837200,NFLX,Tuesday,3.5,4,buy,wait
7,2025-03-12,EA,137.0,138.15,135.73,136.12,3177500,NFLX,Wednesday,3.0,3,wait,wait
8,2025-03-13,EA,135.79,138.26,135.23,137.72,3165100,NFLX,Thursday,3.1666666666666665,3,wait,wait
9,2025-03-14,EA,138.15,139.23,137.87,138.71,3065500,NFLX,Friday,3.0,3,wait,wait


In [166]:
trades = utils.get_class_instance(
    "SentiTrades",
    "./src/strategy/trade/senti_trades.py",
    entry_struct="multiple",
    exit_struct="half_lifo",
    num_lots=10,
    percent_loss=0.01,
    exit_method="nearest_loss",
    monitor_close=False,
)

df_trades, df_pa = trades.gen_trades(df_signal)
df_trades

idx : 0
dt : 2025-03-03 16:00:00-05:00
close : 129.23
ent_sig : wait
ex_sig : wait
net_pos : 0
net_pos after update : 0
len(self.open_trades) : 0
open_trades : []

idx : 1
dt : 2025-03-04 16:00:00-05:00
close : 131.82
ent_sig : wait
ex_sig : wait
net_pos : 0
net_pos after update : 0
len(self.open_trades) : 0
open_trades : []

idx : 2
dt : 2025-03-05 16:00:00-05:00
close : 134.05
ent_sig : buy
ex_sig : wait
net_pos : 0
net_pos after update : 10
len(self.open_trades) : 1
open_trades : 
[
   {
      ticker: 'EA', ent_dt: '2025-03-05', ent_act: 'buy', ent_lots: 10, ent_price: 134.05, ex_dt: None, ex_act: None, ex_lots: 0, ex_price: None
   },
]

idx : 3
dt : 2025-03-06 16:00:00-05:00
close : 136.79
ent_sig : buy
ex_sig : wait
net_pos : 10
stop_price [long] : 132.71 -> monitor low [current: 133.37]
net_pos after update : 20
len(self.open_trades) : 2
open_trades : 
[
   {
      ticker: 'EA', ent_dt: '2025-03-05', ent_act: 'buy', ent_lots: 10, ent_price: 134.05, ex_dt: None, ex_act: None, ex_

,news_ticker,ticker,entry_datetime,entry_action,entry_lots,entry_price,exit_datetime,exit_action,exit_lots,exit_price,days_held,profit_loss,percent_ret,daily_ret,win
0,NFLX,EA,2025-03-05 16:00:00-05:00,buy,10,134.05,2025-03-11 16:00:00-04:00,sell,10,139.03,5,4.98,0.037150,0.007322,1
1,NFLX,EA,2025-03-06 16:00:00-05:00,buy,10,136.79,2025-03-11 16:00:00-04:00,sell,10,139.03,4,2.24,0.016375,0.004069,1
2,NFLX,EA,2025-03-10 16:00:00-04:00,buy,10,140.43,2025-03-11 16:00:00-04:00,sell,10,139.03,1,-1.40,-0.009969,-0.009969,0
3,NFLX,EA,2025-03-11 16:00:00-04:00,buy,10,137.88,2025-03-12 16:00:00-04:00,sell,10,136.50,1,-1.38,-0.010009,-0.010009,0
4,NFLX,EA,2025-03-17 16:00:00-04:00,buy,10,142.9,2025-03-20 16:00:00-04:00,sell,10,141.47,3,-1.43,-0.010007,-0.003347,0
5,NFLX,EA,2025-03-18 16:00:00-04:00,buy,10,141.87,2025-03-20 16:00:00-04:00,sell,10,141.47,2,-0.40,-0.002819,-0.001410,0
6,NFLX,EA,2025-03-19 16:00:00-04:00,buy,10,141.97,2025-03-20 16:00:00-04:00,sell,10,141.47,1,-0.50,-0.003522,-0.003522,0
7,NFLX,EA,2025-03-20 16:00:00-04:00,buy,5,143.25,2025-03-21 16:00:00-04:00,sell,5,144.3,1,1.05,0.007330,0.007330,1
8,NFLX,EA,2025-03-25 16:00:00-04:00,buy,10,144.5,2025-03-27 16:00:00-04:00,sell,10,145.3,2,0.8,0.005536,0.002764,1
9,NFLX,EA,2025-03-24 16:00:00-04:00,buy,3,144.55,2025-03-27 16:00:00-04:00,sell,3,145.3,3,0.75,0.005189,0.001727,1


In [167]:
df_pa

,coint_corr_ticker,open,high,low,close,volume,ticker,day_name,av_rating,median_rating_excl,entry_signal,exit_signal,stop_price,triggered
date,,,,,,,,,,,,,,
2025-03-03,EA,130.38,131.6,128.87,129.23,5209500,NFLX,Monday,3.0,3,wait,wait,NaN,NaN
2025-03-04,EA,129.52,133.35,129.24,131.82,4291000,NFLX,Tuesday,3.0,3,wait,wait,NaN,NaN
2025-03-05,EA,131.6,134.51,131.31,134.05,5218000,NFLX,Wednesday,3.25,4,buy,wait,NaN,NaN
2025-03-06,EA,133.65,137.53,133.37,136.79,3676700,NFLX,Thursday,4.5,5,buy,wait,132.71,0
2025-03-07,EA,136.19,140.42,136.0,140.04,3356800,NFLX,Friday,3.0,3,wait,wait,135.42,0
2025-03-10,EA,139.75,142.97,139.22,140.43,5440900,NFLX,Monday,5.0,5,buy,wait,135.42,0
2025-03-11,EA,140.9,140.9,136.9,137.88,3837200,NFLX,Tuesday,3.5,4,buy,wait,139.03,1
2025-03-12,EA,137.0,138.15,135.73,136.12,3177500,NFLX,Wednesday,3.0,3,wait,wait,136.50,1
2025-03-13,EA,135.79,138.26,135.23,137.72,3165100,NFLX,Thursday,3.1666666666666665,3,wait,wait,NaN,NaN


In [107]:
pos = "long"
percent_loss = 0.01
trades = [
    # (10, 134.05),
    # (5, 136.79),
    # (3, 140.43),
    (10, 142.9),
    (10, 141.87),
    (10, 141.97),
    # (2, 137.88),
    # (10, 144.5),
]
total_lots = sum(trade[0] for trade in trades)
total_invest = sum(trade[0] * trade[1] for trade in trades)

# stop_price = (
#     total_invest * (1 - percent_loss) / total_lots
#     if pos == "long"
#     else total_invest * (1 + percent_loss) / total_lots
# )
# stop_price

stop_list = [
    trade[1] * (1 - percent_loss) if pos == "long" else trade[1] * (1 + percent_loss)
    for trade in trades
]
stop_list

[141.471, 140.4513, 140.5503]

In [103]:
134.05 * (1 - percent_loss)

132.70950000000002

In [53]:
for ent_type, ent, ex in itertools.product(
    ["long_only", "short_only"],
    ["multiple", "multiple_half"],
    ["fifo", "lifo", "half_fifo", "half_lifo", "take_all"],
):
    print(ent_type, ent, ex)

long_only multiple fifo
long_only multiple lifo
long_only multiple half_fifo
long_only multiple half_lifo
long_only multiple take_all
long_only multiple_half fifo
long_only multiple_half lifo
long_only multiple_half half_fifo
long_only multiple_half half_lifo
long_only multiple_half take_all
short_only multiple fifo
short_only multiple lifo
short_only multiple half_fifo
short_only multiple half_lifo
short_only multiple take_all
short_only multiple_half fifo
short_only multiple_half lifo
short_only multiple_half half_fifo
short_only multiple_half half_lifo
short_only multiple_half take_all


In [54]:
a = (
    list(
        itertools.product(
            ["long_only", "short_only"],
            ["multiple", "multiple_half"],
            ["fifo", "lifo", "half_fifo", "half_lifo", "take_all"],
        )
    )
    + list(
        itertools.product(
            ["long_only", "short_only"], ["single"], ["fifo", "half_fifo"]
        )
    )
    + list(
        itertools.product(
            ["long_or_short"], ["multiple", "multiple_half", "single"], ["take_all"]
        )
    )
)
a

[('long_only', 'multiple', 'fifo'),
 ('long_only', 'multiple', 'lifo'),
 ('long_only', 'multiple', 'half_fifo'),
 ('long_only', 'multiple', 'half_lifo'),
 ('long_only', 'multiple', 'take_all'),
 ('long_only', 'multiple_half', 'fifo'),
 ('long_only', 'multiple_half', 'lifo'),
 ('long_only', 'multiple_half', 'half_fifo'),
 ('long_only', 'multiple_half', 'half_lifo'),
 ('long_only', 'multiple_half', 'take_all'),
 ('short_only', 'multiple', 'fifo'),
 ('short_only', 'multiple', 'lifo'),
 ('short_only', 'multiple', 'half_fifo'),
 ('short_only', 'multiple', 'half_lifo'),
 ('short_only', 'multiple', 'take_all'),
 ('short_only', 'multiple_half', 'fifo'),
 ('short_only', 'multiple_half', 'lifo'),
 ('short_only', 'multiple_half', 'half_fifo'),
 ('short_only', 'multiple_half', 'half_lifo'),
 ('short_only', 'multiple_half', 'take_all'),
 ('long_only', 'single', 'fifo'),
 ('long_only', 'single', 'half_fifo'),
 ('short_only', 'single', 'fifo'),
 ('short_only', 'single', 'half_fifo'),
 ('long_or_short

In [55]:
len(a)

27